# Deploy TensorFlow model with AWS SageMaker

In this notebook, we will deploy the tensorflow model developed using AWS SageMaker service.

For this, first create a **SageMaker Notebook instance** with instance type `ml.t2.medium`

- In Permission and encryption, create a new role. Choose specific S3 buckets, then choose **Create role.**

- Amazon SageMaker creates an IAM role named `AmazonSageMaker-ExecutionRole-YYYYMMDDTHHmmSS`

- Copy the **IAM role ARN number**, and paste into a text file to save it temporarily.

### Create and upload Dockerfile with training scripts

- Create a new folder named `docker_folder` in JupyterLab. Create a `Dockerfile` and copy the content of the Dockerfile present in this repo (./Dockerfile). You can modify this as per your requirements.

- Upload `./scripts/train.py` and `Dockerfile` in `docker_folder`.

### Build the container

In [ ]:
% cd ~/SageMaker/docker_folder

To build the Docker container, run the following Docker build command,

In [ ]:
! docker build -t tf-custom-container-test .

## Push the container to Amazon Elastic Container Registry (Amazon ECR)

Use below script for pushing the container created to Amazon ECR

In [ ]:
%%sh

# Specify an algorithm name
algorithm_name=tf-custom-container-test

account=$(aws sts get-caller-identity --query Account —output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest (http://amazonaws.com/$%7Balgorithm_name%7D:latest)“

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly

$(aws ecr get-login --region ${region} —no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Once this container is pushed, it can be used anywhere in the SageMaker environment.

We will call Amazon ECR image of the training container.

In [ ]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = "tf-custom-container-test"
ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

ecr_image

Use the `ecr_image` retrieved from the previous step to configure a SageMaker estimator object.

In [ ]:
import sagemaker

from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

estimator = Estimator(image_name=ecr_image,
    role=get_execution_role(),
    base_job_name='tf-custom-container-test-job',
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge'
)

# start training
estimator.fit()

# deploy the trained model
predictor = estimator.deploy(1, instance_type)

### Clean up resources

Unless, you would want to have several sleepless nights, you should clean up all the resources.

1. Open SageMaker Console, choose the notebook instance, choose Actions and click **Stop**.

2. Once instance has been stopped, choose **Actions** and then **DELETE**.

3. Open the Amazon S3 console and delete the bucket that you created for storing model artifacts and the training dataset.